In [13]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Load the SpaceX dataset
spacex_df = pd.read_csv('spacex_launch_dash.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a dropdown list for launch sites
site_options = [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()]

# Create the layout of the dashboard
app.layout = html.Div([
    # Header of the dashboard
    html.H1('SpaceX Launch Dashboard', style={'textAlign': 'center'}),

    # Dropdown for selecting launch site
    dcc.Dropdown(
        id='site-dropdown',
        options=site_options,
        value='ALL',  # default value
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    # Pie chart for launch success
    dcc.Graph(id='success-pie-chart'),

    # Range slider for selecting payload
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={0: '0', 10000: '10000'},
        value=[0, 10000]  # Default range
    ),

    # Scatter plot for success vs payload
    dcc.Graph(id='success-payload-scatter-chart'),
])

# Callback function for rendering the success-pie-chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Aggregating success/failure counts across all sites
        fig = px.pie(spacex_df, names='Launch Site', 
                     title='Success Launches by Site')
        return fig
    else:
        # Filter data for the selected site and count success vs failure
        site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(site_data, names='class', 
                     title=f'Success vs Failure Launches at {selected_site}')
        return fig

# Callback function for rendering the success-payload scatter chart
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_plot(selected_site, payload_range):
    # Filter by selected payload range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]
    
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version Category',
                     title=f'Success vs Payload Mass for {selected_site if selected_site != "ALL" else "All Sites"}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)